# Tutorial 5 - Generate new aqueous species objects for using in pyDEW
One of the advantages of using the ENKI coder module is you can easily create new models, and adjust their parameters. Coder models have two modes: one where the parameters are fixed, and one where the parameters can be varied at any time. The fixed version is faster, and so the default species in pyDEW are set up this way. However, if you want to adjust the properties of a species, or create a new species, it is easy to generate new coder models. *Note: if you just want to experiment (and obtain logKs) for different complexes, you can create a dummy complex when you define the system. The pyQ3 documentation describes this, but I will also write another tutorial soon.*

First, import the required packages. This includes running code in another notebook to define some methods (z_tutorial5_core.ipynb):

In [1]:
import numpy as np
import dill
import pandas as pd
%run z_tutorial5_core.ipynb
import matplotlib.pyplot as plt
from thermoengine import model, phases

/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Create a coder implementation of the HKF model, in their predictive format.

In [2]:
%run z_tutorial5_HKF_equations.ipynb

Set up the properties of the species we will generate. You can have multiple species, just repeat the block of code below.

The name in [] following phase_params is the name you will use when defining the species. The 'Phase' entry in the dictionary is used to locate the file, and should contain only letters, numbers, and underscores. To include the charge you could use '_p3' or '_n2' for +3 or -2, for example. Adding 'new_' in front makes sure the default DEW species won't be overwritten or get confused.

Here I'm using NaCl$_{(aq)}$ as a demonstration, with the values taken from DEW2019.

In [3]:
phase_params = {}
### REPEAT THIS BLOCK OF CODE (WITH NEW PARAMETERS) TO ADD MORE SPECIES ###
phase_params['NACL(AQ)'] = {'Phase': 'new_NaCl_aq',
                            'Formula': 'NA(1)CL(1)',
                            'G_ref':-92910 * 4.184,
                            'S_ref':28 * 4.184,
                            'v_ref':24.000/10,
                            'C_p_ref':8.508 * 4.184,
                            'a1':0, # Leave as zero if predicting
                            'a2':0, # Leave as zero if predicting
                            'a3':0, # Leave as zero if predicting
                            'a4':0, # Leave as zero if predicting
                            'c1':0, # Leave as zero if predicting
                            'c2':0, # Leave as zero if predicting
                            'omega0':0, # Leave as zero if predicting
                            'complex':1, # Is it a complex? If yes, set to 1, if not set to 0
                            'gas':0,  # Is it a gas? If yes set to to 1, if not set to 0
                            'prd_ac':1, # Predict a and c coefficients? 1 = yes, 0 = no
                            'prd_omega':1, # Predict omega0? 1 = yes, 0 = no
                            'theta':228.0, # Probably don't need to change
                            'Psi':2600.0, # Probably don't need to change
                            'eta':694657.0, # Probably don't need to change
                            'rH':3.082, # Probably don't need to change
                            'z':0, # SET THE CHARGE
                            'T_r':298.15, # Probably don't need to change
                            'P_r':1.0} # Probably don't need to change
### END OF BLOCK ##########################################################


Here you can set the module name, and the working directory (where coder will write lots of C code). The `calib=True` argument tells coder to make a model where the parameters can be changed.

This step will take a little while as the C code is compiled in the background.

In [4]:
mod_name = 'tutorial5_species'
modelDB, codermod = make_custom_database(mod_name,phase_params,working_dir='tutorial5_working', calib=True)

/Users/simonmatthews/repos/pyDEW/tutorials/tutorial5_working
Creating (once only) generic fast model code file string
Creating (once only) generic model calib code template include file string
Creating (once only) generic model calib code template code file string
Creating (once only) generic calib model code file string
Creating include file ...
... done!
Creating code file ...
... done
Writing include file to working directory ...
Writing code file to working directory ...
Writing pyxbld file to working directory ...
writing pyx file to working directory ...
Compiling code and Python bindings ...
Success! Import the module named  tutorial5_species


rm: tmpfile2.pyx: No such file or directory
/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /Users/simonmatthews/repos/pyDEW/tutorials/tutorial5_working/tutorial5_species.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /Users/simonmatthews/.pyxbld/temp.macosx-10.9-x86_64-3.7/pyrex/tutorial5_species.c:701:
In file included from /Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969:
/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_

/Users/simonmatthews/repos/pyDEW/tutorials


Extract the species objects and pickle them for later use. See tutorial 6 for how to import them into another notebook.

In [5]:
output = {}
for phsnm in phase_params:
    iphs = modelDB.get_phase(phsnm)
    output[phsnm] = iphs
with open('tutorial5_species.pkl','wb') as file:
    dill.dump(output, file)
    file.close()